In [2]:
'''데이터 생성 스크립트'''
# bboxes = data['bboxes'] # nested array: [[x, y, w, h],...]
# scores = data['scores'] # [0.6, 0.89,...]
# labels = data['labels'] # [0, 4,...] # yolo 라벨 0-23
# timestamp = data['timestamp'] # unix
# width = data['width'] # 이미지 width
# height = data['height'] # 이미지 height
# region_and_name = data['region_and_name'] # 카메라 정보. id?
# img_directory = data['img_directory'] # 이미지 저장 경로
# dict_values([[[580.052604675293, 217.65965223312378, 60.063819885253906, 58.98589611053467], [330.85792541503906, 175.94241857528687, 33.85063171386719, 39.26307678222656]], [0.6402683258056641, 0.5894748568534851], [0, 4], 1693530106898, 640, 360, '지역', 'C:/cctv_images/지역_1693530106898.jpg'])

'데이터 생성 스크립트'

In [3]:
import pandas as pd

columns = ['cctv_id', 'cctv_name', 'center_name', 'timestamp', 'x', 'y', 'bwidth', 'bheight', 'width', 'height', 'score', 'label', 'img_directory']

df = pd.DataFrame(columns=columns)
df

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory


In [4]:
cctv_df = pd.read_csv(r'static\CCTV.csv')
cctv_df.head()

,Unnamed: 0,RN,CCTVID,CCTVNAME,CENTERNAME,XCOORD,YCOORD,URL
0,0,1,L280034,가덕영업소,거제교통정보센터,128.817344,35.025620,https://www.utic.go.kr/view/map/openDataCctvSt...
1,1,2,L280005,거가1교,거제교통정보센터,128.731926,35.017735,https://www.utic.go.kr/view/map/openDataCctvSt...
2,2,3,L280003,거가1교(저도방향),거제교통정보센터,128.721416,35.017084,https://www.utic.go.kr/view/map/openDataCctvSt...
3,3,4,L280054,거가2교,거제교통정보센터,128.758293,35.014669,https://www.utic.go.kr/view/map/openDataCctvSt...
4,4,5,L280001,거제면 귀목정,거제교통정보센터,128.581956,34.862783,https://www.utic.go.kr/view/map/openDataCctvSt...


In [5]:
cctv_df.shape

(7746, 8)

In [6]:
import random
import time
import os
class GenerateRandomData:
    def __init__(self, x1=None, y1=None, score=None, label=None, width=None, height=None, cctvid=None, img_directory_pre=None):
        '입력 안 하면 랜덤으로 x1, y1, label 설정됨'
        self._width = width
        self._height = height
        self._generate_width_height()
        self.x1 = x1
        self.y1 = y1
        if not all([self.x1, self.y1]):
            self.x1, self.y1 = self.generate_starting_point()
        self.bbox = self.generate_bbox(self.x1, self.y1)
        self.score = score or self.generate_score()
        self.label = label or self.generate_label()
        self._cctvid = cctvid or self._generate_region_and_name()[0]
        self._img_directory_pre = img_directory_pre or self._generate_img_directory_pre()
        self.img_directory = os.path.join(self._img_directory_pre, self._cctvid + '_' + str(int(time.time()*1000)) + '.jpg')

    def generate_starting_point(self):
        x1 = random.uniform(0, self._width)
        y1 = random.uniform(0, self._height)
        return x1, y1

    def generate_bbox(self, x, y):
        '''
        x,y coordinates로 bbox 만들기\n
        x, y, bwidth, bheight로 구성
        '''
        bwidth = random.uniform(10,100)
        bheight = random.uniform(10,100)
        # 이미지 사이즈 안 넘어가게
        if x + bwidth > self._width: bwidth = self._width - x
        if y + bheight > self._height: bheight = self._height - y
        return [x, y, bwidth, bheight]
    
    def generate_score(self):
        'score 생성'
        return random.uniform(0.5, 1)
    
    def generate_label(self):
        'label 생성'
        return random.randrange(0, 23)
    
    def _generate_width_height(self, new=False):
        'width, height 생성'
        self._width = random.choice([640, 1280, 1600, 1920]) if new else (self._width if self._width else random.choice([640, 1280, 1600, 1920]))
        self._height = int(self._width * (9/16)) if new else (self._height if self._height else int(self._width * (9/16)))
        return self._width, self._height
    
    def generate_timestamp(self):
        'timestamp 생성'
        return int(time.time()*1000)

    def _generate_region_and_name(self, new=False):
        'region 정보 생성'
        idx = random.randrange(0, 7746)
        cctv_id, cctv_name, center_name = cctv_df.iloc[idx, 2:5]
        return [cctv_id, cctv_name, center_name]
    
    def _generate_img_directory_pre(self, new=False):
        '파일 부모 저장경로 생성'
        return random.choice([r'C:/cctv_images', r'/home/cctv_images', r'~/Downloads/cctv_images'])
    
    def generate_img_directory(self, timestamp):
        '파일 저장경로 생성'
        return os.path.join(self._img_directory_pre, self._cctvid + '_' + str(timestamp) + '.jpg')
    
    def generate_bulk_data(self, count=100):
        'bulk data 생성'
        dict_ = {column: [] for column in columns}
        # width, height 생성
        width, height = self._generate_width_height(new=True)
        # cctv 정보 생성
        cctv_id, cctv_name, center_name = self._generate_region_and_name(new=True)
        # img_directory_pre 생성
        img_directory_pre = self._generate_img_directory_pre(new=True)
        for _ in range(count):
            # columns = ['bboxes', 'scores', 'labels', 'timestamp', 'width', 'height', 'cctvid', 'region_and_name', 'centername', 'img_directory']
            # x1,y1 생성
            x, y = self.generate_starting_point()
            # bbox 생성
            bbox = self.generate_bbox(x, y)
            # score 생성
            score = self.generate_score()
            # label 생성
            label = self.generate_label()
            # timestamp 생성
            timestamp = self.generate_timestamp()
            # img_directory 생성
            # img_directory = os.path.join(img_directory_pre, cctv_id + '_' + str(timestamp) + '.jpg')
            img_directory = str(timestamp) + '.jpg'
            dict_['cctv_id'].append(cctv_id)
            dict_['cctv_name'].append(cctv_name)
            dict_['center_name'].append(center_name)
            dict_['timestamp'].append(timestamp)
            dict_['x'].append(bbox[0])
            dict_['y'].append(bbox[1])
            dict_['bwidth'].append(bbox[2])
            dict_['bheight'].append(bbox[3])
            dict_['width'].append(width)
            dict_['height'].append(height)
            dict_['score'].append(score)
            dict_['label'].append(label)
            dict_['img_directory'].append(img_directory)
        return pd.DataFrame.from_dict(dict_)

generator = GenerateRandomData()
generator.generate_label()
generator._generate_region_and_name()
df_ = generator.generate_bulk_data()

In [7]:
df = pd.DataFrame(columns=columns)

for _ in range(1000):
    df = pd.concat([df, generator.generate_bulk_data(100)])
df

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory
0,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,677.990656,716.779378,22.040975,99.197316,1920,1080,0.615556,12,1693554726005.jpg
1,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1908.207763,362.974969,11.792237,77.708205,1920,1080,0.632227,18,1693554726005.jpg
2,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1711.775876,727.894918,63.016306,56.569888,1920,1080,0.696284,19,1693554726005.jpg
3,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,790.999098,919.434792,58.191014,13.472484,1920,1080,0.625577,15,1693554726005.jpg
4,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1552.609556,468.121714,40.576633,50.932073,1920,1080,0.507243,21,1693554726005.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,1490.801592,445.568887,14.969024,93.659790,1600,900,0.923155,21,1693554732563.jpg
96,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,328.780362,138.920981,53.016945,92.183735,1600,900,0.908351,0,1693554732563.jpg
97,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,308.862898,580.579357,77.173326,16.662294,1600,900,0.949823,22,1693554732563.jpg
98,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,593.542852,788.957926,24.668465,59.786968,1600,900,0.674014,20,1693554732563.jpg


In [8]:
df.to_csv('generated_data5.csv', index=False)

In [9]:
df_ = pd.read_csv('generated_data5.csv')
df_

,cctv_id,cctv_name,center_name,timestamp,x,y,bwidth,bheight,width,height,score,label,img_directory
0,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,677.990656,716.779378,22.040975,99.197316,1920,1080,0.615556,12,1693554726005.jpg
1,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1908.207763,362.974969,11.792237,77.708205,1920,1080,0.632227,18,1693554726005.jpg
2,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1711.775876,727.894918,63.016306,56.569888,1920,1080,0.696284,19,1693554726005.jpg
3,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,790.999098,919.434792,58.191014,13.472484,1920,1080,0.625577,15,1693554726005.jpg
4,E900063,[국도17호선]진천원덕교차로,국가교통정보센터(국도),1693554726005,1552.609556,468.121714,40.576633,50.932073,1920,1080,0.507243,21,1693554726005.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,1490.801592,445.568887,14.969024,93.659790,1600,900,0.923155,21,1693554732563.jpg
99996,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,328.780362,138.920981,53.016945,92.183735,1600,900,0.908351,0,1693554732563.jpg
99997,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,308.862898,580.579357,77.173326,16.662294,1600,900,0.949823,22,1693554732563.jpg
99998,E912660,[중부내륙선]양평,국가교통정보센터(고속도로),1693554732563,593.542852,788.957926,24.668465,59.786968,1600,900,0.674014,20,1693554732563.jpg
